In [430]:
import os
import sys
import json
import requests

#TO DO - load in authorization vs declare, set as enviroment variable, 

In [436]:
# Reused variables
with open('klarna_api.json') as json_file:
    config = json.load(json_file)
    BASE_URL = config['base_url']
    AUTHORIZATION = (config['user_id'], config['password'])
    HEADERS = config['headers']

In [437]:
# Wrapper methods to help generate usable output
def get_method(url, author=AUTHORIZATION, head=HEADERS):
    output = requests.get(url, auth=author, headers=head,  stream=True)
    output.raise_for_status()
    return json.loads(output.text)

def post_method(url, payload={}, authorize=AUTHORIZATION, head=HEADERS):
    if payload:
        output = requests.post(url, data=json.dumps(payload), auth=authorize, headers=head)
    else:
        output = requests.post(url, auth=authorize, headers=head)
    output.raise_for_status()
    if output.text:
        return json.loads(output.text)
    else:
        return output

In [438]:
instrument_example = {
  "locale": "en-US",
  "purchase_country": "US",
  "purchase_currency": "USD",
  "merchant_reference1": "my_customerorderID",
  "order_amount": 18000,
  "order_tax_amount": 2000,
  "order_lines": [
    {
      "reference": "KLN-100",
      "quantity": 1,
      "unit_price": 8000,
      "total_amount": 8000,
      "type": "instrument",
      "name": "bagpipe"
    },
    {
      "reference": "KLN-101",
      "quantity": 1,
      "unit_price": 8000,
      "total_amount": 8000,
      "type": "instrument",
      "name": "drums"
    },
    {
      "quantity": 1,
      "total_amount": 2000,
      "type": "sales_tax",
      "name": "Tax",
      "unit_price": 2000
    }
  ]
}

In [433]:
klarna_example = {
  "locale": "en-US",
  "purchase_country": "US",
  "purchase_currency": "USD",
  "merchant_reference1": "Klarna_customerorderID",
  "order_amount": 18000,
  "order_tax_amount": 2000,
  "order_lines": [
    {
      "reference": "KLN-100",
      "quantity": 1,
      "unit_price": 8000,
      "image_url": "https://www.klarna.com/example/image/prod.jpg",
      "total_amount": 8000,
      "type": "physical",
      "product_url": "https://www.klarna.com/example/widget1=prod",
      "name": "Klarna Widget 1"
    },
    {
      "reference": "KLN-101",
      "quantity": 1,
      "unit_price": 8000,
      "image_url": "https://www.klarna.com/example/image/prod.jpg",
      "total_amount": 8000,
      "type": "physical",
      "product_url": "https://www.klarna.com/example/widget2=prod",
      "name": "Klarna Widget 2"
    },
    {
      "quantity": 1,
      "total_amount": 2000,
      "type": "sales_tax",
      "name": "Tax",
      "unit_price": 2000
    }
  ]
}

CREATE A SESSION

In [439]:
# Create a new session
session = post_method('{}/payments/v1/sessions'.format(BASE_URL),  
                       payload=klarna_example)

In [440]:
# Realize we can get the client token right here, but want to work on get method
session_id = session['session_id']

In [441]:
# GET SESSION
session = get_method('{}/payments/v1/sessions/{}'.format(BASE_URL, session_id))

In [442]:
session

{'purchase_country': 'us',
 'purchase_currency': 'usd',
 'locale': 'en-US',
 'order_amount': 18000,
 'order_tax_amount': 2000,
 'order_lines': [{'type': 'physical',
   'reference': 'KLN-100',
   'name': 'Klarna Widget 1',
   'quantity': 1,
   'unit_price': 8000,
   'total_amount': 8000,
   'total_discount_amount': 0,
   'product_url': 'https://www.klarna.com/example/widget1=prod',
   'image_url': 'https://www.klarna.com/example/image/prod.jpg'},
  {'type': 'physical',
   'reference': 'KLN-101',
   'name': 'Klarna Widget 2',
   'quantity': 1,
   'unit_price': 8000,
   'total_amount': 8000,
   'total_discount_amount': 0,
   'product_url': 'https://www.klarna.com/example/widget2=prod',
   'image_url': 'https://www.klarna.com/example/image/prod.jpg'},
  {'type': 'sales_tax',
   'name': 'Tax',
   'quantity': 1,
   'unit_price': 2000,
   'total_amount': 2000,
   'total_discount_amount': 0}],
 'merchant_urls': {},
 'merchant_reference1': 'Klarna_customerorderID',
 'design': 'klarna',
 'status

In [444]:
session['client_token']

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjgyMzA1ZWJjLWI4MTEtMzYzNy1hYTRjLTY2ZWNhMTg3NGYzZCJ9.ewogICJzZXNzaW9uX2lkIiA6ICI0NGI2Y2Q3Ni0yZjc4LTIyYWEtOTE0MC03ZDA2ZTA1YzhlNWQiLAogICJiYXNlX3VybCIgOiAiaHR0cHM6Ly9rbGFybmEtcGF5bWVudHMtbmEucGxheWdyb3VuZC5rbGFybmEuY29tL3BmbXZwIiwKICAiZGVzaWduIiA6ICJrbGFybmEiLAogICJsYW5ndWFnZSIgOiAiZW4iLAogICJwdXJjaGFzZV9jb3VudHJ5IiA6ICJVUyIsCiAgInRyYWNlX2Zsb3ciIDogZmFsc2UsCiAgImVudmlyb25tZW50IiA6ICJwbGF5Z3JvdW5kIiwKICAibWVyY2hhbnRfbmFtZSIgOiAiUGxheWdyb3VuZCBEZW1vIE1lcmNoYW50IiwKICAic2Vzc2lvbl90eXBlIiA6ICJQQVlNRU5UUyIsCiAgImNsaWVudF9ldmVudF9iYXNlX3VybCIgOiAiaHR0cHM6Ly9uYS5wbGF5Z3JvdW5kLmtsYXJuYWV2dC5jb20iLAogICJzY2hlbWUiIDogdHJ1ZSwKICAiZXhwZXJpbWVudHMiIDogWyB7CiAgICAibmFtZSIgOiAiaW4tYXBwLXNkay1jYXJkLXNjYW5uaW5nIiwKICAgICJ2YXJpYXRlIiA6ICJjYXJkLXNjYW5uaW5nLWVuYWJsZSIsCiAgICAicGFyYW1ldGVycyIgOiB7CiAgICAgICJ2YXJpYXRlX2lkIiA6ICJjYXJkLXNjYW5uaW5nLWVuYWJsZSIKICAgIH0KICB9IF0KfQ.KBGOgZWP7TN54xDH36sa6SQ1c2ljfE10R9l1dJas4s6zCdSDwM9jQ87Pi_DvilBMLzJ8JOBnrz3GxrucyDYHe6jcZrRNO6fPV-fWCqMwZ8O

UPDATE STATUS:

In [359]:
status_update = post_method('{}/payments/v1/sessions/{}'.format(BASE_URL, session_id), 
                              payload=instrument_purchase)

HTTPError: 400 Client Error: Bad Request for url: https://api-na.playground.klarna.com/payments/v1/sessions/03b5bb24-10aa-2cae-80c6-9ada0f1acd97

In [354]:
status_update.text

'<html>\r\n<head><title>401 Authorization Required</title></head>\r\n<body>\r\n<center><h1>401 Authorization Required</h1></center>\r\n<hr><center>openresty</center>\r\n</body>\r\n</html>\r\n'

In [136]:
%%HTML
<script>
  window.klarnaAsyncCallback = function () {

     Klarna.Payments.init({
  client_token: 
})

  };
</script>
<script src="https://x.klarnacdn.net/kp/lib/v1/api.js" async></script>

In [139]:
%%js

Klarna.Payments.init({
  client_token: 
})

<IPython.core.display.Javascript object>

In [141]:
%%js
try {
Klarna.Payments.init({
    client_token: 
} catch (e) {

}

<IPython.core.display.Javascript object>

Getting the SDK to work- after realizing the session creation was the problem 
VERY exited to see '''show_form: true'''

![title](Happy_when_it_works.png)

ORDERS MANAGEMENT:

CREATE ORDER:

In [445]:
AUTHORIZATION_TOKEN = '' # Token from the client side, used to generate the 
# 'c3b196b6-27b1-26f3-b8ef-ef1c000aa5a1'
# "f0bd1b8b-1d72-246d-b05e-02248e6571df"
# '83b749e4-c0bb-24d3-92e3-049b8256d170'
# '62b46a9c-cabd-29d3-b5bc-a7113a625159'

In [446]:
order_create = post_method('{}/payments/v1/authorizations/{}/order'.format(BASE_URL, AUTHORIZATION_TOKEN),
                          payload=klarna_example)

In [447]:
order_create

{'order_id': '7cb66274-4354-2147-b75a-af1ad3d3ff44',
 'fraud_status': 'ACCEPTED',
 'authorized_payment_method': {'type': 'slice_it_by_card'}}

CANCEL ORDER (Note can not be done once order is captured)

In [301]:
order_canceled = requests.post('https://api-na.playground.klarna.com/ordermanagement/v1/orders/{}/cancel'.format('52bcaca0-35bb-2c78-90b4-baa99199cd80'), 
                       auth=('N104701_f720721d7d66', 'uEWwK2Ziu2pZKdk1'), 
                       headers=HEADERS)

In [409]:
order_create['order_id']

'08b266c9-93c9-20b1-80a9-c01be286c2b0'

GET ORDER

In [378]:
get_order = get_method('{}/ordermanagement/v1/orders/{}'.format(BASE_URL, order_create['order_id']))

ACKNOWLEDGE ORDER

In [385]:
acknowledge_order = post_method('{}/ordermanagement/v1/orders/{}/acknowledge'.format(BASE_URL, 
                                                                                    order_create['order_id']))

CAPTURE ORDER

In [387]:
capture_payload = {
    "captured_amount": 18000,
    "description": "captured",
    "reference": "for this id"
#     "order_lines": [
#         {
#             "reference": "75001",
#             "type": "physical",
#             "quantity": 1,
#             "quantity_unit": "pcs.",
#             "name": "string",
#             "total_amount": 0,
#             "unit_price": 0,
#             "total_discount_amount": 0,
#             "tax_rate": 0,
#             "total_tax_amount": 0,
#             "merchant_data": "Some metadata",
#             "product_url": "https://yourstore.example/product/headphones",
#             "image_url": "https://yourstore.example/product/headphones.png",
#             "product_identifiers": {
#                 "category_path": "Electronics Store > Computers & Tablets > Desktops",
#                 "global_trade_item_number": "735858293167",
#                 "manufacturer_part_number": "BOXNUC5CPYH",
#                 "brand": "Intel"
#             }
#         }
#     ],
#     "shipping_info": [
#         {
#             "shipping_company": "DHL US",
#             "shipping_method": "Home",
#             "tracking_number": "63456415674545679874",
#             "tracking_uri": "http://shipping.example/findmypackage?63456415674545679874",
#             "return_shipping_company": "DHL US",
#             "return_tracking_number": "93456415674545679888",
#             "return_tracking_uri": "http://shipping.example/findmypackage?93456415674545679888"
#         }
#     ],
#     "shipping_delay": 0
}


In [410]:
capture_order = post_method('{}/ordermanagement/v1/orders/{}/captures'.format(BASE_URL,
                                                                              order_create['order_id']),
                                                                              payload=capture_payload)

In [411]:
capture_order

<Response [201]>

REFUND ORDER:

In [414]:
refund_info = {
    "refunded_amount": 400
}

In [415]:
refunded = post_method('{}/ordermanagement/v1/orders/{}/refunds'.format(BASE_URL, order_create['order_id']),
                        payload=refund_info)

In [416]:
refunded

<Response [201]>